# Module 3: Filtering & Enhancement

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/computer-graphics/blob/main/notebooks/03_Filtering_Enhancement.ipynb)

**Week 5-6: Image Filtering & Enhancement Techniques**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Topics
- Convolution fundamentals
- Gaussian Blur
- Median Filter
- Bilateral Filter
- Image Sharpening

**Key Functions**: `cv2.filter2D()`, `cv2.GaussianBlur()`, `cv2.medianBlur()`, `cv2.bilateralFilter()`